In [ ]:
import http.client
import json
from types import SimpleNamespace
import pandas as pd
from urllib.parse import quote
pd.set_option('display.max_colwidth', None)
!pip install xlsxwriter

# Org - Specific Stuff

In [ ]:
url_to_query = "twapi-demo-dev-ed.develop.my.salesforce.com"
# This is the GDrive folder to save things
gdrive_folder = "PFI TW API/"
#fileName = 'Register a Client English'
fileName = 'proof-of-concept-Hannah_v2.xlsx'

# Just run all the cells below this, only change them if you know what you're doing/enjoy frustration. After running, scroll to the last section to view any errors.

# Load a File from GDrive

In [ ]:
from google.colab import drive
# Mount Google Drive so we can pull in files
drive.mount('/content/drive')
# Setup some directory names (helps to keep work separate)
workingDirectory = '/content/drive/MyDrive/' + gdrive_folder

In [ ]:
!ls "{workingDirectory}"

# Import the excel file into SalesForce

## TODO - refactor into shared library

In [ ]:
def get_oauth_token_from_login():
    conn = http.client.HTTPSConnection(url_to_query)
    payload = ''
    headers = {}
    #NOTE: This is bad security practice, normally we would want these login details to be hidden so they don't get backed up to git
    #For simplicity of this demo, we're exposing details here
    #TODO: before production usage, separate these variables into a JSON file that's loaded from Google Drive
    login_url = "/services/oauth2/token?grant_type=password&client_id=xxx&client_secret=yyy&username=www&password=aaa"
    conn.request("POST",  login_url, payload, headers)
    res = conn.getresponse()
    data = res.read()
    decoded_form_data = data.decode("utf-8")
    data_obj = json.loads(decoded_form_data, object_hook=lambda d: SimpleNamespace(**d))
    print(data_obj)
    return "OAuth " + data_obj.access_token

In [ ]:
auth_header = get_oauth_token_from_login()
print(auth_header)

In [ ]:
def get_version_changelog_from_form_name(form_name):
    form_name_urlsafe = quote(form_name, safe='/')
    form_endpoint = "/services/apexrest/formdata/v1?objectType=GetFormData&name=" + form_name_urlsafe
    form_dataframe = get_pandas_dataframe_from_json_web_call(form_endpoint)
    try:
      form_id = form_dataframe.id[0]
      form_external_id = form_dataframe.externalId[0]
    except: 
      print('No form matches that name')
      return '', '', '','', None
    # Note - this script assumes there is only 1 form matching the name
    json_form_version = form_dataframe.formVersion[0]
    form_version_string = str(json_form_version[0]).replace('\'','"')
    form_version_json_obj = json.loads(form_version_string)
    form_version_id = form_version_json_obj['versionid']
    changelog_number = form_version_json_obj['changeLogNumber']
    print('Form Version ID: ',form_version_id, ' Form ID: ', form_id, ' Changelog: ', changelog_number, ' externalID: ',form_external_id)
    return form_version_id, changelog_number, form_id, form_external_id, form_dataframe

In [ ]:
def get_pandas_dataframe_from_json_web_call(endpoint_to_hit):
    conn = http.client.HTTPSConnection(url_to_query)
    payload = ''
    headers = {
      'Authorization': auth_header,
      'Content-Type': 'application/json',
    }
    conn.request("GET",  endpoint_to_hit, payload, headers)
    res = conn.getresponse()
    data = res.read()
    decoded_form_data = data.decode("utf-8")
    data_obj = json.loads(decoded_form_data)
    records_dataframe = pd.json_normalize(data_obj, record_path =['records'])
    return records_dataframe

In [ ]:
def upload_payload_to_url(endpoint_to_upload, payload):
      conn = http.client.HTTPSConnection(url_to_query)
      headers = {
        'Authorization': auth_header,
        'Content-Type': 'application/json',
      }
      conn.request("PUT", endpoint_to_upload, payload.encode(), headers)
      res = conn.getresponse()
      data = res.read()
      decoded_form_data = data.decode("utf-8")
      data_obj = json.loads(decoded_form_data)
      results_dataframe = pd.json_normalize(data_obj)
      print(data.decode("utf-8"))
      return results_dataframe

## Read excel file

In [ ]:
# Read excel file into dataframes
xls = pd.ExcelFile(workingDirectory + fileName)
upload_form_dataframe = pd.read_excel(xls, 'Forms',header=1)
upload_questions_without_options = pd.read_excel(xls, 'Questions', header=1)
upload_options = pd.read_excel(xls, 'Options', header=1)
upload_question_mapping = pd.read_excel(xls, 'Question_Mappings', header=1)
upload_field_mapping_no_question_mapping = pd.read_excel(xls, 'Field_Mappings', header=1)
upload_skip_logic = pd.read_excel(xls, 'Skip_Logic', header=1)
upload_orm = pd.read_excel(xls, 'Object_Relationship_Mappings', header=1)

In [ ]:
upload_questions_without_options

## Form

In [ ]:
upload_form_dataframe_relevant_columns = upload_form_dataframe[['name','alias','messageAfterSubmission','description']].fillna("")
form_name_to_upload = str(upload_form_dataframe_relevant_columns.name[0])
# Fetch latest form ID and changelog from the API
form_version_id, changelog_number, form_id, form_external_id, form_dataframe = get_version_changelog_from_form_name(form_name_to_upload)

In [ ]:
upload_form_dataframe_relevant_columns

In [ ]:
updating_existing_form = form_id
if updating_existing_form:
      #update existing form
      print('Update existing form ' + form_id)
      upload_form_dataframe_relevant_columns['id'] = form_id
      formVersionString = '"formVersion": [{"versionid": "' + form_version_id + '","changeLogNumber": "' + changelog_number + '"}]'
else:
      #create new form
      print('Creating new form')
      upload_form_dataframe_relevant_columns['id'] = ""
      formVersionString = '"formVersion": [{"versionid": "","changeLogNumber": ""}]'

# Forms made on the UI will have a null external ID even if they're updating, so handle form external IDs outside of "is updating" logic
if (form_external_id):
    upload_form_dataframe_relevant_columns['externalId'] = form_external_id
else:
    #Use the name as the external ID if none specified
    upload_form_dataframe_relevant_columns['externalId'] = upload_form_dataframe_relevant_columns['name']
      


#This will only upload 1 form
upload_str = str(upload_form_dataframe_relevant_columns.T.astype(str).to_json(force_ascii=False)).replace('{"0":','{"records":[')[:-2] + ',' + formVersionString + '}]}'
print(upload_str)
form_update_endpoint = '/services/apexrest/formdata/v1?objectType=PutFormData'
payload = upload_str
form_result = upload_payload_to_url(form_update_endpoint, upload_str)

## Fetch existing Questions

In [ ]:
# First read in all known questions + options for this form into dataframes
question_endpoint = "/services/apexrest/questiondata/v1?objectType=GetQuestionData&formVersionId=" + form_version_id
question_dataframe = pd.DataFrame(columns=['externalId', 'id', 'name', 'caption', 'cascadingLevel',\
       'cascadingSelect', 'controllingQuestion', 'displayRepeatSectionInTable',\
       'dynamicOperation', 'dynamicOperationTestData', 'dynamicOperationType',\
       'exampleOfValidResponse', 'form', 'formVersion', 'hidden', 'maximum',\
       'minimum', 'parent', 'position', 'previousVersion', 'printAnswer',\
       'repeatSourceValue', 'repeatTimes', 'required', 'responseValidation',\
       'showAllQuestionOnOnePage', 'skipLogicBehavior', 'skipLogicOperator',\
       'hint', 'testDynamicOperation', 'type', 'useCurrentTimeAsDefault',\
       'changeLogNumber', 'options'])
question_dataframe = pd.concat([question_dataframe,get_pandas_dataframe_from_json_web_call(question_endpoint)])
# TODO - refactor to shared
#Iterate all questions that have options and create a new dataframe that has just the options
options_dataframe = pd.DataFrame(columns=["externalId" , "id" , "name" , "position" , "caption" ])
for index, frame in question_dataframe.iterrows():
    if (frame.options):
      questionId = frame.id
      individual_option_df = pd.read_json(str(frame.options).replace('\'','"'))
      individual_option_df['questionId'] = questionId
      options_dataframe = pd.concat([individual_option_df,options_dataframe])

questions_without_options = question_dataframe.loc[:, question_dataframe.columns != 'options']
#if updating_existing_form:
existing_questions_lookup = questions_without_options[['externalId','id','name']]
existing_options_lookup = options_dataframe[['externalId','id','name']]

In [ ]:
existing_questions_lookup

## Format new questions/options

### For this script, assume the excel file has been unsquished (only has 1 column with a caption)

In [ ]:
upload_options_sanitized = upload_options.copy()
for column in upload_options:
    if ('caption' in column):
        #Remove the language-specific suffix
        upload_options_sanitized = upload_options_sanitized.rename(columns={column:'caption'})

# Treat externalID as optional; if it is not defined, use the name instead
if (upload_options_sanitized.empty):
    upload_options_sanitized['externalId'] = None
else:
    upload_options_sanitized['externalId'] = upload_options_sanitized['name']
upload_options_sanitized = upload_options_sanitized[['name','position','caption','questionName','externalId']]

In [ ]:
upload_questions_sanitized = upload_questions_without_options.copy().fillna("")
for column in upload_questions_sanitized:
    if ('caption' in column):
        #Remove the language-specific suffix
        upload_questions_sanitized = upload_questions_sanitized.rename(columns={column:'caption'})

In [ ]:
upload_options_sanitized = upload_options_sanitized.merge(existing_options_lookup[['id','name']],how="left",on="name").fillna("")

In [ ]:
# Fetch latest form ID and changelog from the API
form_version_id, changelog_number, form_id, form_external_id, form_dataframe = get_version_changelog_from_form_name(form_name_to_upload)

In [ ]:
#Convert options df back into nested json
options_associated_with_questions = upload_options['questionName'].drop_duplicates()
options_associated_with_questions
upload_questions_with_options = upload_questions_sanitized.copy()
upload_questions_with_options['options'] = None

# Treat externalID as optional; if it is not defined, use the name instead
upload_questions_with_options['externalId'] = upload_questions_with_options['name']

In [ ]:
upload_questions_with_options = upload_questions_with_options.merge(existing_questions_lookup[['id','name']],how="left",on="name").fillna('')

In [ ]:
upload_questions_with_options

In [ ]:
for questionName in options_associated_with_questions.iteritems():
      thisQuestionName = questionName[1]
      upload_options_json = str(upload_options_sanitized[upload_options_sanitized['questionName'] == thisQuestionName][['externalId','id','name','position','caption']].to_json(orient='records',force_ascii=False))
      print(upload_options_json)
      row_index = upload_questions_with_options.index[upload_questions_with_options['name'] == thisQuestionName ].tolist()[0]
      print(row_index)
      upload_questions_with_options.at[row_index,'options']= upload_options_json

In [ ]:
upload_questions_with_options = upload_questions_with_options.merge( \
        existing_questions_lookup[['id','name']].rename(columns={'id':'parentId','name':'parentName'}),
        how="left",on="parentName")\
        .rename(columns={'parentId':'parent'}).fillna('')

In [ ]:
# In cases where the parent name is not blank but the parent is blank, this means that no ID already exists for the question parent (it hasn't been updated) - use the externalID
parent_externalId_lookup = upload_questions_with_options[upload_questions_with_options['parentName'] == ''][['name','externalId']].rename(columns={'name':'parentName','externalId':'parentExternalId'})

In [ ]:
upload_questions_with_options = upload_questions_with_options.merge(parent_externalId_lookup,how="left",on='parentName').fillna('')
upload_questions_with_options['parent'] = upload_questions_with_options.apply(lambda x: str(x.parentExternalId) if x.parentName and not x.parent else x.parent, axis = 1)


In [ ]:
upload_questions_with_options['form'] = form_id
upload_questions_with_options['formVersion'] = form_version_id
upload_questions_with_options['changeLogNumber'] = changelog_number

In [ ]:
upload_questions_with_options

In [ ]:
question_with_options_creation_string = '{"records":' + str(upload_questions_with_options[['externalId', 'id', 'name',
       'caption', 'cascadingLevel',
       'cascadingSelect', 'controllingQuestion', 'displayRepeatSectionInTable',
       'dynamicOperationType', 'exampleOfValidResponse',
       'form', 'formVersion', 'hidden', 
       'maximum',  'minimum', 'parent', 'position',
       'previousVersion', 'printAnswer',
       'repeatSourceValue', 'repeatTimes',
       'required', 'responseValidation', 'showAllQuestionOnOnePage',
       'skipLogicBehavior', 'skipLogicOperator', 'hint',
       'testDynamicOperation', 'type', 'useCurrentTimeAsDefault',
       'changeLogNumber', 'options']].to_json(orient="records",force_ascii=False)).replace('\\','').replace('"[{"','[{"').replace('"}]"','"}]').replace(',"options":""',',"options":[]').replace('null','""') + '}'
question_with_options_creation_string

In [ ]:
questions_result = upload_payload_to_url('/services/apexrest/questiondata/v1?objectType=PutQuestionData', question_with_options_creation_string)

## Read Back Questions and update any relevant IDs

In [ ]:
question_endpoint = "/services/apexrest/questiondata/v1?objectType=GetQuestionData&formVersionId=" + form_version_id
questions_after_upload = get_pandas_dataframe_from_json_web_call(question_endpoint)

In [ ]:
questions_after_upload

In [ ]:
question_id_lookup = questions_after_upload[['id','name']].rename(columns={'id':'questionId','name':'questionName'})

### Update any dependent objects from the spreadsheet:


*   Question Mapping "question" field
*   Field Mapping "repeat" field
*   Skip Logic "sourceQuestion" and "parentQuestion"  

In [ ]:
question_mapping_referencing_new_ids = upload_question_mapping.merge(question_id_lookup,how="left").rename(columns={"questionId":'question'})

In [ ]:
contains_repeat_sections = upload_field_mapping_no_question_mapping[pd.isna(upload_field_mapping_no_question_mapping['repeatQuestionName']) == False]

In [ ]:
if (not contains_repeat_sections.empty):
    field_mapping_referencing_new_ids = upload_field_mapping_no_question_mapping.merge(question_id_lookup,how="left",left_on='repeatQuestionName',right_on="questionName").rename(columns={'questionId':'repeat'})
else:
    upload_field_mapping_no_question_mapping['repeat'] = ''
    field_mapping_referencing_new_ids = upload_field_mapping_no_question_mapping

In [ ]:
upload_skip_logic_referencing_new_ids = upload_skip_logic.merge(question_id_lookup,how="left",left_on="sourceQuestionName",right_on="questionName")\
    .rename(columns={"questionId":"sourceQuestion"})\
    .drop(columns=['sourceQuestionName','questionName'])\
    .merge(question_id_lookup,how="left",left_on="parentQuestionName",right_on="questionName")\
    .rename(columns={"questionId":"parentQuestion"}).drop(columns=['parentQuestionName','questionName'])

## Read existing Field and Form Mappings

In [ ]:
field_mapping_endpoint = "/services/apexrest/formmappingdata/v1?objectType=GetFormMappingData&formVersionId=" + form_version_id
field_mapping_dataframe = pd.DataFrame(columns = ['externalId', 'id', 'name', 'form', 'formVersion',\
       'formVersionMappingField', 'mobileUserField', 'objectApiName',\
       'formMappingField', 'isReference', 'matchingField', 'repeat',\
       'submissionAPIField', 'changeLogNumber', 'questionMappings'])
field_mapping_dataframe = pd.concat([field_mapping_dataframe,get_pandas_dataframe_from_json_web_call(field_mapping_endpoint)])
#Iterate all form mappings that have question mappings and create a new dataframe that has just the question mappings
question_mapping_dataframe = pd.DataFrame(columns=["externalId", "name", "id", "fieldAPIName","isBroken","question","scoringGroup"])
for index, frame in field_mapping_dataframe.iterrows():
    if (frame.questionMappings):
      print(str(frame.questionMappings).replace('\'','"'))
      field_mapping_id = frame.id
      #JSON is case-sensitive, python apparently converts it into uppercase
      individual_question_mapping_df = pd.read_json(str(frame.questionMappings).replace('\'','"').replace("True","true").replace("False","false"))
      individual_question_mapping_df['field_mapping_id'] = field_mapping_id
      question_mapping_dataframe = pd.concat([individual_question_mapping_df,question_mapping_dataframe])
field_mapping_without_questions = field_mapping_dataframe.loc[:, field_mapping_dataframe.columns != 'questionMappings']

## Field and Form Mapping

In [ ]:
form_version_id, changelog_number, form_id, form_external_id, form_dataframe = get_version_changelog_from_form_name(form_name_to_upload)

In [ ]:
#Convert options df back into nested json
question_mapping_associated_with_field_mapping = question_mapping_referencing_new_ids['fieldMappingName'].drop_duplicates()

In [ ]:
upload_field_mapping_with_question_mapping = field_mapping_referencing_new_ids.copy()
upload_field_mapping_with_question_mapping['questionMappings'] = None

In [ ]:
if (upload_field_mapping_with_question_mapping.empty):
    upload_field_mapping_with_question_mapping['externalId'] = None
else:
    upload_field_mapping_with_question_mapping['externalId'] = upload_field_mapping_with_question_mapping['name']

In [ ]:
if (question_mapping_referencing_new_ids.empty):
    question_mapping_referencing_new_ids['externalId'] = None
else:
    question_mapping_referencing_new_ids['externalId'] = question_mapping_referencing_new_ids['name']

In [ ]:
upload_question_mapping_with_ids = question_mapping_referencing_new_ids.merge(question_mapping_dataframe[['name','id']],how="left",on="name")

In [ ]:
for field_mapping_name in question_mapping_associated_with_field_mapping.iteritems():
      thisFieldMapping = field_mapping_name[1]
      print(thisFieldMapping)
      upload_question_mapping_json = str(upload_question_mapping_with_ids[upload_question_mapping_with_ids['fieldMappingName'] == thisFieldMapping][['externalId', 'name', 'id', 'fieldAPIName', 'isBroken','question', 'scoringGroup']].to_json(orient='records',force_ascii=False))
      print(upload_question_mapping_json)
      row_index = upload_field_mapping_with_question_mapping.index[upload_field_mapping_with_question_mapping['name'] == thisFieldMapping ].tolist()[0]
      print(row_index)
      upload_field_mapping_with_question_mapping.at[row_index,'questionMappings']= upload_question_mapping_json

In [ ]:
upload_field_mapping_with_question_mapping['form'] = form_id
upload_field_mapping_with_question_mapping['formVersion'] = form_version_id
upload_field_mapping_with_question_mapping['changeLogNumber'] = changelog_number

In [ ]:
upload_field_mapping_with_question_mapping = upload_field_mapping_with_question_mapping.merge(field_mapping_without_questions[['id','name']],how="left",on="name")

In [ ]:
upload_field_mapping_with_question_mapping = upload_field_mapping_with_question_mapping.fillna("")

In [ ]:
upload_field_mapping_string = '{"records":' + str(upload_field_mapping_with_question_mapping[['externalId', 'id', 'name', 'form', 'formVersion',
       'formVersionMappingField', 'mobileUserField', 
       'objectApiName', 'formMappingField', 
       'isReference', 'matchingField', 'repeat', 
       'submissionAPIField', 'changeLogNumber', 'questionMappings']].astype(str).to_json(orient="records",force_ascii=False)).replace('\\','')\
       .replace('"[{"','[{"').replace(']"}',']}') + "}"
print(upload_field_mapping_string)

In [ ]:
if (upload_field_mapping_with_question_mapping.empty):
    form_mapping_result = "No Form Mapping to upload"
else:
    form_mapping_result = upload_payload_to_url('/services/apexrest/formmappingdata/v1?objectType=PutFormMappingData', upload_field_mapping_string)

## Read back field and form mapping IDs, update ORM IDs

In [ ]:
field_mapping_endpoint = "/services/apexrest/formmappingdata/v1?objectType=GetFormMappingData&formVersionId=" + form_version_id
field_mapping_after_upload_dataframe = pd.DataFrame(columns = ["externalId","id" ,"name" ,"form" ,"formVersion" ,"formVersionMappingField" ,"mobileUserField" ,"patScoreMappingField" ,"objectApiName" ,"formMappingField" ,"intervieweeMapField" ,"isReference" ,"matchingField" ,"repeat" ,"useAsInterviewee" ,"submissionAPIField" ,"changeLogNumber" ,"questionMappings" ])
field_mapping_after_upload_dataframe = pd.concat([field_mapping_after_upload_dataframe,get_pandas_dataframe_from_json_web_call(field_mapping_endpoint)])

In [ ]:
upload_orm_with_replaced_id = upload_orm.merge(field_mapping_after_upload_dataframe.rename(columns={"name":"parentSurveyName","id":"parentSurveyMapping"})[['parentSurveyName','parentSurveyMapping']],how="left",on="parentSurveyName")\
    .merge(field_mapping_after_upload_dataframe.rename(columns={"name":"childSurveyName","id":"childSurveyMapping"})[['childSurveyName','childSurveyMapping']],how="left",on="childSurveyName")\
    [['name','fieldApiName','parentSurveyMapping','childSurveyMapping']]

if (upload_orm_with_replaced_id.empty):
    upload_orm_with_replaced_id['externalId'] = None
else:
    upload_orm_with_replaced_id['externalId'] = upload_orm_with_replaced_id['name']


## Update ORM

In [ ]:
# Fetch latest form ID and changelog from the API
form_version_id, changelog_number, form_id, form_external_id, form_dataframe = get_version_changelog_from_form_name(form_name_to_upload)

In [ ]:
#Fetch existing ORM
orm_endpoint = "/services/apexrest/questiondata/v1?objectType=GetObjectRelationshipMappingData&formVersionId=" + form_version_id
orm_dataframe = pd.DataFrame(columns=["externalId" ,"id" ,"name" ,"fieldApiName" ,"parentSurveyMapping" ,"childSurveyMapping" ,"formVersion" ,"changeLogNumber"])
orm_dataframe = pd.concat([orm_dataframe, get_pandas_dataframe_from_json_web_call(orm_endpoint)])

In [ ]:
upload_orm_with_replaced_id = upload_orm_with_replaced_id.merge(orm_dataframe[['id','name']],how="left",on="name")

In [ ]:
upload_orm = upload_orm_with_replaced_id[['externalId', 'id', 'name', 'fieldApiName',
       'parentSurveyMapping', 'childSurveyMapping']].fillna("")

upload_orm['formVersion'] = form_version_id
upload_orm['changeLogNumber'] = changelog_number

In [ ]:
string_to_upload = '{"records":' + upload_orm.astype(str).to_json(orient="records",force_ascii=False) + '}'
string_to_upload

In [ ]:
orm_update_endpoint = '/services/apexrest/objectrelationshipmappingdata/v1?objectType=PutObjectRelationshipMappingData'
if (upload_orm.empty):
    orm_result = "No ORM to Upload"
else:
    orm_result = upload_payload_to_url(orm_update_endpoint, string_to_upload)

## Upload Skip Logic

In [ ]:
# Fetch latest form ID and changelog from the API
form_version_id, changelog_number, form_id, form_external_id, form_dataframe = get_version_changelog_from_form_name(form_name_to_upload)

In [ ]:
#Fetch existing Skip Logic
skip_logic_endpoint = "/services/apexrest/questiondata/v1?objectType=GetSkipLogicData&formVersionId=" + form_version_id
skip_logic_dataframe = pd.DataFrame(columns=["externalId" ,"id" ,"negate" ,"skipValue" ,"condition" ,"parentQuestion" ,"sourceQuestion" ,"form" ,"formVersion" ,"changeLogNumber"])
skip_logic_dataframe = pd.concat([skip_logic_dataframe, get_pandas_dataframe_from_json_web_call(skip_logic_endpoint)])

In [ ]:
skip_logic_dataframe['joinColumn'] = skip_logic_dataframe['parentQuestion'] + '-' + skip_logic_dataframe['sourceQuestion']

In [ ]:
upload_skip_logic_referencing_new_ids['joinColumn'] = upload_skip_logic_referencing_new_ids['parentQuestion'] + '-' + upload_skip_logic_referencing_new_ids['sourceQuestion']

In [ ]:
# Get existing IDs and external IDs for any existing skip logic
upload_skip_logic_referencing_new_ids_joined = upload_skip_logic_referencing_new_ids.merge(skip_logic_dataframe[['id','externalId','joinColumn']],how="left",on="joinColumn").fillna("")


In [ ]:
if (upload_skip_logic_referencing_new_ids_joined.empty):
    upload_skip_logic_referencing_new_ids_joined['externalId'] = None
else:
    upload_skip_logic_referencing_new_ids_joined['externalId'] = upload_skip_logic_referencing_new_ids_joined.apply(lambda x: str(x['externalId']) if x['externalId'] else x['joinColumn'], axis=1)

In [ ]:
upload_skip_logic = upload_skip_logic_referencing_new_ids_joined[['externalId', 'id', 'negate', 'skipValue', 'condition',
       'parentQuestion', 'sourceQuestion']]

upload_skip_logic['form'] = form_id
upload_skip_logic['formVersion'] = form_version_id
upload_skip_logic['changeLogNumber'] = changelog_number

In [ ]:
string_to_upload = '{"records":' + upload_skip_logic.astype(str).to_json(orient="records",force_ascii=False) + '}'
string_to_upload

In [ ]:
form_update_endpoint = '/services/apexrest/skiplogicdata/v1/?objectType=PutSkipLogicData'
if (upload_skip_logic.empty):
    skip_logic_result = "No Skip Logic to Upload"
else:
    skip_logic_result = upload_payload_to_url(form_update_endpoint, string_to_upload)

# Review any errors

In [ ]:
form_result

In [ ]:
questions_result

In [ ]:
form_mapping_result

In [ ]:
orm_result

In [ ]:
# NOTE: Bug IDALMSA-12051 causes the API to return "Skip Condition created successfully" when the API has actually updated instead of created. Low priority to fix as this doesn't break anything.
skip_logic_result